# Descrizione notebook
In questo notebook si fa utilizzo dell'Item-Based Collaborative Filtering per raccomandare una barzelletta ad un utente.

In [1]:
# import delle librerie necessarie
import numpy as np
import pandas as pd
from itertools import combinations
import math

In [2]:
# lettura dei rating normalizzati dal relativo CSV
normalized_ratings_df = pd.read_csv('./data/normalized_ratings.csv')

# lettura dei rating non normalizzati dal relativo CSV
no_normalized_ratings_df = pd.read_csv('./data/no_normalized_ratings.csv')

# lettura delle barzellette dal relativo CSV
jokes_df = pd.read_csv('./data/jokes.csv')

# Similarità tra item: Adjusted Coseno Similarità

In [3]:
def cosine_similarity(x, y): 
    t1, t2, t3 = 0, 0, 0 
    for i, j in zip(x, y):        
        t1+=i*j # numeratore
        t2+=i*i # denominatore_1
        t3+=j*j # denominatore_2
    return t1 / (np.sqrt(t2) * np.sqrt(t3))

# Predizione

In [4]:
def score_user_item_ibcf(item_target, items_rated, active_user_rating, similarity_matrix):
    t1, t2, score = 0, 0, 0
    for i in items_rated:
        try:
            sim = float(similarity_matrix.loc[((similarity_matrix['elem_1'] == item_target) & (similarity_matrix['elem_2'] == i))\
                                        | ((similarity_matrix['elem_1'] == i) & (similarity_matrix['elem_2'] == item_target))]["similarity"])
            t1+= sim * float(active_user_rating[i]) # numeratore
            t2+= sim # denominatore
        except: 
            continue
    if t2 != 0:
        score = t1 / t2
    return score

# Raccomandazione

In [5]:
# si costruisce la matrice di similarità, ovvero una lista di coppie di item, per 
# ogni coppia si specifica la similarità (ad -1 a 1)
all_jokes = (list(normalized_ratings_df.columns))[2:]
jokes_pairs = list(combinations(all_jokes, 2))

similarity_matrix = []

for i_1, i_2 in jokes_pairs:
    app = normalized_ratings_df[[i_1, i_2]].dropna()
    similarity_matrix.append([i_1, i_2, cosine_similarity(app[i_1], app[i_2])])
    
similarity_matrix = pd.DataFrame(similarity_matrix, columns = ['elem_1', 'elem_2', 'similarity'])
similarity_matrix = similarity_matrix[similarity_matrix['similarity'] > 0.1]
similarity_matrix

,elem_1,elem_2,similarity
0,joke_1,joke_2,0.202696
1,joke_1,joke_3,0.173774
8,joke_1,joke_10,0.159604
35,joke_1,joke_37,0.122071
42,joke_1,joke_44,0.121290
...,...,...,...
4938,joke_95,joke_99,0.154420
4940,joke_96,joke_97,0.307388
4942,joke_96,joke_99,0.153141
4945,joke_97,joke_99,0.146586


In [6]:
def recommendation_ibcf(active_user_id, no_normalized_ratings_df, similarity_matrix, val):
    # si prende la riga relativa all'utente con l'id specificato
    active_user = no_normalized_ratings_df[no_normalized_ratings_df['user_id'] == active_user_id]
    # si considerano solo i rating espressi dall'utente target
    active_user_rating = active_user.iloc[:, 2:]
    
    # si prendono le barzellette raccomandabili all'utente
    recommendation_columns = []
    for column in active_user_rating.columns:
        if math.isnan(active_user_rating[column].values[0]):
            recommendation_columns.append(column)
    
    # si prendono le barzellette già valutate dall'utente
    rated_items = list(set(list(active_user_rating.columns)) - set(recommendation_columns))
    # si fa la raccomandazione
    top_score = -np.inf
    joke_to_suggest = ''
    final = []
    for joke in recommendation_columns:
        score = score_user_item_ibcf(joke, rated_items, active_user_rating, similarity_matrix)
        if score > top_score:
            top_score = score
            joke_to_suggest = joke
        app_2 = {}
        app_2["user_id"] = active_user_id
        app_2[joke] = score
        final.append(app_2)
        
    if (not(val)):
        print("L'utente target è: ")
        print(active_user)
        print("Le barzellette raccomandabili all'utente target sono:")
        print(recommendation_columns)
        for elem in final:
            print("per la barzelletta " + str(list(elem.keys())[1]) + " si è predetto un punteggio di: " + str(elem[list(elem.keys())[1]]))
        print('\nil valore più alto predetto è ' + str(top_score) + ' su 10')
        print('quindi la barzelletta da raccomandare per la quale si è predetto quel punteggio è: ', joke_to_suggest) 
        s = jokes_df[jokes_df["joke_id"] == int(joke_to_suggest[5:])]["joke"]
        for c in s: 
            print(c)
    
    return final
    

In [7]:
final = recommendation_ibcf(1001, no_normalized_ratings_df, similarity_matrix, False)

L'utente target è: 
      user_id  number_of_jokes_rated  joke_1  joke_2  joke_3  joke_4  joke_5  \
1000     1001                     53     NaN     NaN     NaN     NaN    4.76   

      joke_6  joke_7  joke_8  ...  joke_91  joke_92  joke_93  joke_94  \
1000    4.71    4.66    0.68  ...      NaN      NaN      NaN      NaN   

      joke_95  joke_96  joke_97  joke_98  joke_99  joke_100  
1000      NaN      NaN      NaN      NaN      NaN       NaN  

[1 rows x 102 columns]
Le barzellette raccomandabili all'utente target sono:
['joke_1', 'joke_2', 'joke_3', 'joke_4', 'joke_9', 'joke_23', 'joke_24', 'joke_33', 'joke_37', 'joke_43', 'joke_47', 'joke_55', 'joke_57', 'joke_58', 'joke_60', 'joke_64', 'joke_67', 'joke_71', 'joke_72', 'joke_73', 'joke_74', 'joke_75', 'joke_76', 'joke_77', 'joke_78', 'joke_79', 'joke_80', 'joke_81', 'joke_82', 'joke_83', 'joke_84', 'joke_85', 'joke_86', 'joke_87', 'joke_88', 'joke_89', 'joke_90', 'joke_91', 'joke_92', 'joke_93', 'joke_94', 'joke_95', 'joke_96', '